In [ ]:
import numpy as np
from torch.distributions import Categorical
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [ ]:
# model = torch.nn.Sequential(
#         torch.nn.Linear(18, 36),
#         torch.nn.ReLU(),
#         torch.nn.Linear(36, 54),
#         torch.nn.ReLU(),
#         torch.nn.Linear(54, 6),
#         torch.nn.Softmax()
#         )

model = torch.nn.Sequential(
        torch.nn.Linear(18, 54),
        torch.nn.ReLU(),
        torch.nn.Linear(54, 6),
        torch.nn.Softmax()
        )

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Обучение не делать неправильные ходы

In [ ]:
# Отображает поле
def print_board(board):
  for row in range(3):
    for col in range(6):
      if board[row][col] == -1:
        k = "  "
      else: k = "   "
      print(board[row][col], end=k)
    print("\n")

# Записывает заполненные столбцы
def check_valid_columns(board):
  rez = []
  for col in range(6):
    if all(board[row][col] != 0 for row in range(3)):
      rez.append(col)
  return rez

# Считает победы
def calculate(board, player):
  count = 0
  # Горизонтальные
  for row in range(3):
    for col in range(4):
      if board[row][col] == player and board[row][col+1] == player and board[row][col+2] == player:
        count += 1
  # Вертикальные
  for col in range(6):
    for row in range(1):
      if board[row][col] == player and board[row+1][col] == player and board[row+2][col] == player:
        count += 1
  # Диагонали (слева направо)
  for row in range(1):
    for col in range(4):
      if board[row][col] == player and board[row+1][col+1] == player and board[row+2][col+2] == player:
        count += 1
  # Диагонали (справа налево)
  for row in range(1, 3):
    for col in range(4):
      if board[row][col] == player and board[row-1][col+1] == player and board[row-2][col+2] == player:
        count += 1

  return count

# Делает ход
def action(board, turn, education=True):
  all_cols = [0, 1, 2, 3, 4, 5]
  wrong_cols = check_valid_columns(board)
  available_cols = list(set(all_cols)-set(wrong_cols))
  if turn == 1:
    vector = [val for row in board for val in row]
    space = torch.tensor(vector, dtype=torch.float32)
    probs = model(space)

    m = Categorical(probs)

    if education: #для обучения
      x = m.sample()
    else: #для игры
      x = torch.argmax(probs)

    log_prob = m.log_prob(x)

    return x, log_prob
  else: # болван
    return np.random.choice(available_cols), None

def get_reward(board, player, invalid_move=False):

    if invalid_move:
      reward = -100
    else:
      reward = 1

    return np.float32(reward)

In [ ]:
gamma = 1
alpha = 0.9
player1_wins = 0
player2_wins = 0
ww = 0
max_iter = 30000
count = 0
count_ = 0

model = torch.load("yurchenko.pt")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for episode in range(max_iter):

  board = [[0] * 6 for x in range(3)]
  turn = np.random.choice([-1, 1])

  log_probs = []
  rewards = []

  for round in range(18):
    # Действие
    if turn == 1:
      move, log_prob = action(board, turn)
      log_probs.append(log_prob)
    else:
      move, _ = action(board, turn)

    # Отражение действия на доске
    for pos in range(2, -1, -1):
      if board[pos][move] == 0:
        board[pos][move] = turn
        inv = False
        break
      else:
        inv = True

    #Получение награды
    if turn == 1:
      reward = get_reward(board, 1, invalid_move=inv)
      rewards.append(reward)

    if inv:
      count += 1
      round -= 1
    else:
      turn *= -1

    if len(check_valid_columns(board)) == 6:
      break

  ###
  loss = 0.
  for r in range(len(rewards)):
    R = 0.
    for i in range(r, len(rewards)):
      R += (gamma**(i - r))*rewards[i]

    loss += -alpha*R*log_probs[r]

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  player1_scores = calculate(board, 1)
  player2_scores = calculate(board, -1)

  if player1_scores > player2_scores:
    player1_wins +=1
  elif player1_scores < player2_scores:
    player2_wins +=1
  else:
    ww += 1

  if (episode + 1) % (max_iter / 20) == 0:
    print(f"Episode {episode+1}: Побед нейронной сети {player1_wins}, Побед болванчика: {player2_wins}, Ничьи: {ww}, Ошибки на n-итераций: {count-count_}")
    count_ = count

torch.save(model, f'yurchenko.pt')
print(f"Количество неверных ходов: {count}")

<ipython-input-23-69c768c7258a>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("zalupaE_220k_100k.pt")


Episode 1500: Побед нейронной сети 1371, Побед болванчика: 44, Ничьи: 85, Ошибки на n-итераций: 1
Episode 3000: Побед нейронной сети 2652, Побед болванчика: 134, Ничьи: 214, Ошибки на n-итераций: 5
Episode 4500: Побед нейронной сети 3936, Побед болванчика: 213, Ничьи: 351, Ошибки на n-итераций: 3
Episode 6000: Побед нейронной сети 5270, Побед болванчика: 266, Ничьи: 464, Ошибки на n-итераций: 1
Episode 7500: Побед нейронной сети 6559, Побед болванчика: 335, Ничьи: 606, Ошибки на n-итераций: 0
Episode 9000: Побед нейронной сети 7827, Побед болванчика: 418, Ничьи: 755, Ошибки на n-итераций: 1
Episode 10500: Побед нейронной сети 9145, Побед болванчика: 485, Ничьи: 870, Ошибки на n-итераций: 3
Episode 12000: Побед нейронной сети 10491, Побед болванчика: 529, Ничьи: 980, Ошибки на n-итераций: 2
Episode 13500: Побед нейронной сети 11823, Побед болванчика: 593, Ничьи: 1084, Ошибки на n-итераций: 1
Episode 15000: Побед нейронной сети 13108, Побед болванчика: 653, Ничьи: 1239, Ошибки на n-итера

Обучение побеждать

In [ ]:
def get_reward_(board, player, invalid_move=False):

    wins = calculate(board, player)
    loses = calculate(board, -player)
    reward = (wins - loses) * 10

    if wins == loses:
        reward = -5

    if invalid_move:
      reward = -100

    return np.float32(reward)

gamma = 1
alpha = 0.9
player1_wins = 0
player2_wins = 0
ww = 0
max_iter = 100000
count = 0
count_ = 0
wins = 0
loses = 0
ww_ = 0

model = torch.load("yurchenko.pt")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for episode in range(max_iter):

  board = [[0] * 6 for x in range(3)]
  turn = np.random.choice([-1, 1])

  log_probs = []
  rewards = []

  for round in range(18):
    # Действие
    if turn == 1:
      move, log_prob = action(board, turn)
      log_probs.append(log_prob)
    else:
      move, _ = action(board, turn)

    # Отражение действия на доске
    for pos in range(2, -1, -1):
      if board[pos][move] == 0:
        board[pos][move] = turn
        inv = False
        break
      else:
        inv = True

    #Получение награды
    if turn == 1:
      reward = get_reward_(board, 1, invalid_move=inv)
      rewards.append(reward)

    if inv:
      count += 1
      round -= 1
    else:
      turn *= -1

    if len(check_valid_columns(board)) == 6:
      break

  ###
  loss = 0.
  for r in range(len(rewards)):
    R = 0.
    for i in range(r, len(rewards)):
      R += (gamma**(i - r))*rewards[i]

    loss += -alpha*R*log_probs[r]

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  player1_scores = calculate(board, 1)
  player2_scores = calculate(board, -1)

  if player1_scores > player2_scores:
    player1_wins +=1
  elif player1_scores < player2_scores:
    player2_wins +=1
  else:
    ww += 1

  if (episode + 1) % (max_iter / 20) == 0:
    print(f"Episode {episode+1}: Побед нейронной сети {player1_wins - wins}, Побед болванчика: {player2_wins - loses}, Ничьи: {ww-ww_}, Ошибки: {count-count_}")
    count_ = count
    wins = player1_wins
    loses = player2_wins
    ww_ = ww

torch.save(model, f'yurchenko.pt')
print(f"Количество неверных ходов: {count}")


<ipython-input-8-5e9dd2da8daf>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("zalupaE_200k.pt")


Episode 5000: Побед нейронной сети 4076, Побед болванчика: 473, Ничьи: 451, Ошибки: 119
Episode 10000: Побед нейронной сети 4667, Побед болванчика: 110, Ничьи: 223, Ошибки: 60
Episode 15000: Побед нейронной сети 4714, Побед болванчика: 101, Ничьи: 185, Ошибки: 37
Episode 20000: Побед нейронной сети 4643, Побед болванчика: 136, Ничьи: 221, Ошибки: 43
Episode 25000: Побед нейронной сети 4693, Побед болванчика: 103, Ничьи: 204, Ошибки: 19
Episode 30000: Побед нейронной сети 4670, Побед болванчика: 121, Ничьи: 209, Ошибки: 26
Episode 35000: Побед нейронной сети 4682, Побед болванчика: 107, Ничьи: 211, Ошибки: 20
Episode 40000: Побед нейронной сети 4632, Побед болванчика: 131, Ничьи: 237, Ошибки: 36
Episode 45000: Побед нейронной сети 4657, Побед болванчика: 132, Ничьи: 211, Ошибки: 28
Episode 50000: Побед нейронной сети 4672, Побед болванчика: 121, Ничьи: 207, Ошибки: 37
Episode 55000: Побед нейронной сети 4734, Побед болванчика: 73, Ничьи: 193, Ошибки: 25
Episode 60000: Побед нейронной се

Тест на болванчике

In [ ]:
test_model = torch.load("yurchenko.pt")
player1_wins = 0
player2_wins = 0
ww = 0


def action_2(board, turn):
  all_cols = [0, 1, 2, 3, 4, 5]
  wrong_cols = check_valid_columns(board)
  available_cols = list(set(all_cols)-set(wrong_cols))
  if turn == 1:
    vector = [val for row in board for val in row]
    space = torch.tensor(vector, dtype=torch.float32)
    probs = test_model(space)
    x = torch.argmax(probs)
    return x
  else: # болван
    return np.random.choice(available_cols)

count = 0

for episode in range(10000):

  board = [[0] * 6 for x in range(3)]
  turn = np.random.choice([-1, 1])

  for round in range(18):
    if turn == 1:
      move = action_2(board, turn)
      inv = True
    else:
      move = action_2(board, turn)

    for pos in range(2, -1, -1):
      if board[pos][move] == 0:
        board[pos][move] = turn
        inv = False
        break

    if inv:
      # print_board(board)
      # print(move)
      count += 1

    turn *= -1

  player1_scores = calculate(board, 1)
  player2_scores = calculate(board, -1)

  if player1_scores > player2_scores:
    player1_wins +=1
  elif player1_scores < player2_scores:
    player2_wins +=1
  else:
    ww += 1

print(f"Побед нейронной сети: {player1_wins}, Побед болванчика: {player2_wins}, Ничьи: {ww}")
print(f"Ходы не туда: {count}")

<ipython-input-5-f5dfe154dacc>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_model = torch.load("yurchenko.pt")


Побед нейронной сети: 9276, Побед болванчика: 271, Ничьи: 453
Ходы не туда: 0
